# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# APK Files to load
#file_paths = (
#     "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.0.63-2076_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.5.72-2077_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.0.66-2078_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.7.78-2085_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_11.0.0.72-2086_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#    "../../../Downloads/com.snapchat.android_11.0.1.72-2089_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#    "../../../Downloads/com.snapchat.android_11.0.6.82-2095_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
#)


file_paths = (
# #     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726452_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726384_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168935_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168922_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
    
# #     "../../../Downloads/com.instagram.android_159.0.0.29.122-244390467_minAPI23(arm64-v8a)(560,640dpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_159.0.0.29.122-244390338_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"

     "../../../Downloads/com.instagram.android_159.0.0.40.122-245196070_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
     "../../../Downloads/com.instagram.android_159.0.0.40.122-245196047_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_159.0.0.40.122-245196070_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_159.0.0.40.122-245196047_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `named_c_decs` in form of a tuple of Strings
* `named_m_decs` in form of a tuple of MethodDeclarations
* `named_f_decs` in form of a tuple of FieldDeclarations

In [11]:
named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.6Mq',
    'COMMENT_LONGCLICK_HANDLER': 'X.05e',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.3A4',
    'DM_HOLDER': 'X.4wn',
    'DM_REPLY': 'X.3qU',
    'DM_VOICEMESSAGE': 'X.4jE',
    'DMVIDEOPLAYER': 'X.5WN',
    'FEEDITEM': 'X.1Ue',
    'FRAGMENT': 'X.1Pf',
    'HEADERGENERATOR': 'X.0S1',
    'HTTPREQUESTTASKFACTORY': 'X.0pI',
    'INAPP_NOTIFICATION': 'X.5tv',
    'IGTV_VIEWER_FRAGMENT': 'X.3oy',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.11y',
    'JACKSON_CREATOR': 'X.0is',
    'LIVE_POST': 'X.23p',
    'MAINFEED': 'X.1QB',
    'MEDIAITEM': 'X.1Ui',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1mM',
    'NOTIFICATION_HANDLER': 'X.2U4',
    'PHOTOTIMERCONTROLLER': 'X.2to',
    'POSTDETAIL': 'X.1cs',
    'POSTGESTURE_SIMPLEPOST': 'X.1vD',
    'POSTGESTURE_CAROUSELIMAGE': 'X.2Ap',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2Rh',
    'POSTVIEW': 'X.1cx',
    'PROFILE': 'X.0lJ',
    'PROFILEDETAIL': 'X.2kw',
    'PROFILEPIC_LONGCLICK': 'X.2lA',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2F0',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0nI',
    'NOTIFICATION_RUNNABLE': 'X.5tw',
    'SERIALSCHEDULER': 'X.0ij',
    'SESSION': 'X.0Lz',
    'SIMPLE_EXECUTOR': 'X.41q',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1VJ',
    'VIDEOSETTINGSMANAGER': 'X.1fq',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [12]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.6Mq', '<init>', 'X.206', 'X.6Nr', 'X.0Lz', 'android.view.View', 'X.6Ma', 'X.6Mj'),
    'COMMENT_ONLONGCLICK': MethodDec('X.05e', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1Ga', 'androidx.fragment.app.FragmentActivity', 'X.0Lz', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Lz'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Lz'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3A4', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.4wn', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.3qU', 'A0K', 'X.3qU', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.3qU', 'A0H', 'X.3qU'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.5WN', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1Ue', 'A00', 'X.0jA'),
    'FRAGMENT_GETACTIVITY': MethodDec('X.1Pf', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0S1', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0pI', 'A03', 'java.lang.String', 'X.0Lz'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.5tv', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.3oy', 'A00', 'X.3oy'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.3oy', 'A0C', 'X.3oy'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.11y', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0is', 'A04', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0is', '<init>', 'X.ExG'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.23p', 'A00', 'X.0k1', 'X.23q'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'AKp'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1QB', 'configureActionBar', 'X.1Ki'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1Ui', 'A01', 'X.0jA', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0k1', 'X.1Ui'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1mM', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2U4', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.2to', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1cs', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.1vD', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.2Ap', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2Rh', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1cx', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.2kw', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0lJ', 'A04', 'X.0Lz', 'X.0jo'),
    'PROFILE_ONLONGCLICK': MethodDec('X.2lA', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2F0', 'A0U', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1G', 'X.1tM'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0m', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0S', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0w', 'X.1o3', 'X.1tM'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0nI', 'A00', 'X.0nH', 'X.10u'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.5tw', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0ij', 'A02', 'X.0nl'),
    'SESSION_GETTOKEN': MethodDec('X.0Lz', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.41q', 'onFail', 'X.2Bb'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.41q', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.41q', '<init>', 'X.0Lz'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1VJ', 'A03', 'X.0k1', 'X.0jn'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1fq', 'A00', 'X.0Lz'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [13]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1Kh', 'A07'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.206', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.05e', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1W4', 'A0Y'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3A4', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.4wo', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.4xC', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.4jE', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1Ue', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0pv', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.3oy', 'A0E'),
    'MAINFEED_SESSION': FieldDec('X.1QB', 'A0T'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1UL', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1tM', 'A09'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1tM', 'A08'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.11m', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1hU', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.5tv', 'A00'), FieldDec('X.5tw', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.5tv', 'A01'), FieldDec('X.5tw', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1ry', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1s1', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.2l2', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.2l2', 'A00'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.2l2', 'A01'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.2l2', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.2l7', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.11l', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.2ug', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1L'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1hT', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1tM', 'A0D')
}

In [14]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [15]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [16]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [17]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [18]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.1mM#A01() -> X.1mc#A01()
+ Found single candidate for Method. Considering it a match 
	X.0S1#A00(android.content.Context) -> X.0SB#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0w(X.1o3, X.1tM) -> com.instagram.reels.fragment.ReelViewerFragment#A0w(X.1oL, X.1tf)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.inst

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [19]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

.. Considering ambiguous match by UniqueString as match with a majority of 0.89 (LX/3oy; -> LX/3pU;)
+ Found single candidate for Class. Considering it a match! 
	LX/0Lz; -> LX/0Lz;
+ Found single candidate for Class. Considering it a match! 
	LX/0ij; -> LX/0iv;
+ Found single candidate for Class. Considering it a match! 
	LX/0pI; -> LX/0pU;
+ Found single candidate for Class. Considering it a match! 
	LX/0pv; -> LX/0q7;
+ Found single candidate for Class. Considering it a match! 
	LX/11l; -> LX/11x;
+ Found single candidate for Class. Considering it a match! 
	LX/11m; -> LX/11y;
+ Found single candidate for Class. Considering it a match! 
	LX/11y; -> LX/12A;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [20]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/6Mq; -> LX/6Hy;
+ Found single candidate for Class. Considering it a match! 
	LX/05e; -> LX/33H;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3A4; -> LX/3AZ;
+ Found single candidate for Class. Considering it a match! 
	LX/4wn; -> LX/4on;
+ Found single candidate for Class. Considering it a match! 
	LX/5WN; -> LX/6a7;
+ Found single candidate for Class. Considering it a match! 
	LX/0is; -> LX/0j4;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/2to; -> LX/2uF;
+ Found single candidate for Class. Considering it a match! 
	LX/1vD; -> LX/1vW;
* Found 

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [21]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.2Ap#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2Rh#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	X.6Mq#<init>(X.206, X.6Nr, X.0Lz, android.view.View, X.6Ma, X.6Mj) -> X.6Hy#<init>(X.20P, X.6YJ, X.0Lz, android.view.View, X.6Hl, X.5py)
+ Found single candidate for Method. Considering it a match 
	X.05e#onLongPress(android.view.MotionEvent) -> X.33H#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1Ga, androidx.fragment.app.FragmentActivity, X.0Lz, android.os.Bundle) -> com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#lo

+ Found single candidate for Method. Considering it a match 
	X.0Lz#getToken() -> X.0Lz#getToken()
+ Found single candidate for Method. Considering it a match 
	X.1VJ#A03(X.0k1, X.0jn) -> X.1VZ#A03(X.0kD, X.0jz)
Could resolve 0 new Classes, 8 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [22]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0jn; -> LX/0jz;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1XA; -> LX/1XQ;
+ Found single candidate for Class. Considering it a match! 
	LX/23q; -> LX/249;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0py; -> LX/0qA;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a match!

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [23]:
print(len(accumulator.matching_cs), "/", len(named_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

74 / 46 Classes were matched
56 / 60 Methods were matched
37 / 39 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1mM -> X.1mc
• X.0S1 -> X.0SB
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.3qU -> X.3r0
• X.0nI -> X.0nU
• X.1QB -> X.1QR
• X.1fq -> X.1g6
• X.2F0 -> X.2FJ
• X.2kw -> X.2lN
• X.41q -> X.41J
• X.5tv -> X.5ti
• X.5tw -> X.5tj
• X.0Lz -> X.0Lz
• X.0ij -> X.0iv
• X.0pI -> X.0pU
• X.0pv -> X.0q7
• X.11l -> X.11x
• X.11m -> X.11y
• X.11y -> X.12A
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Kh -> X.1Kx
• X.1Pf -> X.1Pv
• X.1Ue 

• com.instagram.reels.fragment.ReelViewerFragment#A0m(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String) -> com.instagram.reels.fragment.ReelViewerFragment#A0m(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String)
• com.instagram.reels.fragment.ReelViewerFragment#A0S(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0S(com.instagram.reels.fragment.ReelViewerFragment)
• X.0ij#A02(X.0nl) -> X.0iv#A02(X.0nx)
• X.41q#<init>(X.0Lz) -> X.41J#<init>(X.0Lz)
• com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent) -> com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent)
• com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int) -> com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int)
• X.3A4#onLongPress(android.view.MotionEvent) -> X.3AZ#onLongPress(android.view.MotionEvent)
• X.1Ue#A00(X.0jA) -> X.1Uu#A00(X.0jM)
• X.1Pf#

In [24]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(named_c_decs.values(), r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.6Hy', '<init>', 'X.20P', 'X.6YJ', 'X.0Lz', 'android.view.View', 'X.6Hl', 'X.5py'),
    MethodDec('X.33H', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1Gn', 'androidx.fragment.app.FragmentActivity', 'X.0Lz', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Lz),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Lz),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerCont

In [25]:
"""
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"
"""


c_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"


In [26]:
%autoreload

c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Lz)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Lz)
No matching Method found for X.2Ap#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2Rh#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.1ry#A06
No matching Field found for X.1s1#A02


In [27]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.6Hy")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.33H")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.3AZ")

    override val DM_HOLDER = ClassDec("X.4on")

    override val DM_REPLY = ClassDec("X.3r0")

    override val DM_VOICEMESSAGE = ClassDec("X.4js")

    override val DMVIDEOPLAYER = ClassDec("X.6a7")

    override val FEEDITEM = ClassDec("X.1Uu")

    override val FRAGMENT = ClassDec("X.1Pv")

    override v

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1Kx"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A07")

    @FieldClass(["X.20P"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.33H"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1WK"])
    override val COMMENT_TEXT = VariableDec<String>("A0Y")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    @FieldClass(

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```